# Creating the Qdrant database
### Instructions:
1. Start "Docker Desktop"
2. Start "Qdrant" in terminal (wsl for windows):
 
    ```docker run -p 6333:6333 -v $(pwd)/qdrant_storage:/qdrant/storage qdrant/qdrant```
    
    Wait for ascii-print-out. 
3. Run notebook

In [3]:
# Taken from https://qdrant.tech/articles/neural-search-tutorial/

# Import client library
import numpy as np
import json
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

qdrant_client = QdrantClient(host='localhost', port=6333)

collection_names = ['e1', 'e2']

In [4]:
for collection_name in collection_names:
    qdrant_client.recreate_collection(
        collection_name=collection_name, 
        vectors_config=VectorParams(size=768, distance=Distance.COSINE),
    )

    # payload is now an iterator over startup data
    # payload = map(json.loads, fd)
    with open(f'{collection_name}.json', 'r', encoding='utf-8') as infile:
        payload = json.loads(infile.read())

    # Here we load all vectors into memory, numpy array works as iterable for itself.
    # Other option would be to use Mmap, if we don't want to load all data into RAM
    vectors = np.load(f'./vectors_{collection_name}.npy')

    qdrant_client.upload_collection(
    collection_name=collection_name,
    vectors=vectors,
    payload=payload,
    ids=None,  # Vector ids will be assigned automatically
    batch_size=256  # How many vectors will be uploaded in a single request?
    )

/var/folders/rj/vn874pf96jjcpwpn70z_mb1c0000gn/T/ipykernel_1232/855182034.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


In [ ]:
# print(neural_searcher.search("<b>Paris</b> [franskt utt. pari], Frankrikes hufvudstad, näst London Europas folkrikaste stad, ligger under 2° 20' 15\" ö. lgd samt 48° 50' 11,2\" n. br. (nationalobservatoriet), på båda sidor om Seine,"))
# print(neural_searcher.search("<b>Fantasi</b> (Grek. <i>fantasia,</i> af <i>fantazein,</i> göra synbar), föreställningsförmåga, inbillningskraft; diktningsgåfva; inbillning, infall, nyck, hugskott; feberdröm; musikstycke utan bestä"))